<a href="https://colab.research.google.com/github/punit0087/temp/blob/master/Covid19IndiaTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
from math import sin, cos, sqrt, atan2, radians

def get_distance_between_lats_lons(lat1,lon1,lat2,lon2):
# approximate radius of earth in km
        R = 6373.0

        lat1 = radians(lat1)
        lon1 = radians(lon1)
        lat2 = radians(lat2)
        lon2 = radians(lon2)

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c
        return(distance)

In [0]:
file= 'IndiaPostalCodes.csv'
city_wise_coordinates= pd.read_csv(file)
city_wise_coordinates['City'] = city_wise_coordinates['City'].str.upper()
city_wise_coordinates['District'] = city_wise_coordinates['District'].str.upper()
city_wise_coordinates['State'] = city_wise_coordinates['State'].str.upper()
city_wise_coordinates

,Country,PIN,City,State,State_ID,District,District_ID,Community,Lat,Lng,Loc_accuracy
0,IN,744301,MUS,ANDAMAN & NICOBAR ISLANDS,1,NICOBAR,638.0,Carnicobar,9.2333,92.7833,4
1,IN,744301,KAKANA,ANDAMAN & NICOBAR ISLANDS,1,NICOBAR,638.0,Carnicobar,9.1167,92.8000,4
2,IN,744301,SAWAI,ANDAMAN & NICOBAR ISLANDS,1,NICOBAR,638.0,Carnicobar,7.5166,93.6031,4
3,IN,744301,CARNICOBAR,ANDAMAN & NICOBAR ISLANDS,1,NICOBAR,638.0,Carnicobar,9.1833,92.7667,3
4,IN,744301,LAPATHY,ANDAMAN & NICOBAR ISLANDS,1,NICOBAR,638.0,Carnicobar,9.1833,92.7667,3
...,...,...,...,...,...,...,...,...,...,...,...
155565,IN,508224,ARVAPALLY,TELANGANA,40,SURYAPET,696.0,Arvapally,17.4019,79.5407,3
155566,IN,508224,JAJIREDDY GUDEM,TELANGANA,40,SURYAPET,696.0,Arvapally,17.4019,79.5407,3
155567,IN,508279,NAGARAM,TELANGANA,40,SURYAPET,696.0,NAGARAM,18.1460,79.2386,4
155568,IN,508279,PHANIGIRI,TELANGANA,40,SURYAPET,696.0,PHANIGIRI,18.1460,79.2386,3


In [0]:
district_wise_pin_states= city_wise_coordinates.groupby('District')['PIN','State'].agg(pd.Series.mode)
district_wise_lat_lng= city_wise_coordinates.groupby('District')['Lat','Lng'].agg(pd.Series.mean)

district_wise_data_geonames= district_wise_pin_states.merge(district_wise_lat_lng,left_on='District', right_on='District', how= 'inner').reset_index()
#district_wise_data_geonames

In [0]:
'''
Govt_Pincode_data= pd.read_csv('Locality_village_pincode_final_mar_2017.csv',  encoding="ISO-8859-1")

Govt_Pincode_data.rename(columns={'Village':'City'}, inplace=True)
Govt_Pincode_data.rename(columns={'Districtname':'District'}, inplace=True)
Govt_Pincode_data.rename(columns={'StateName':'State'}, inplace=True)
Govt_Pincode_data.rename(columns={'Pincode':'PIN'}, inplace=True)

Govt_Pincode_data['City'] = Govt_Pincode_data['City'].str.upper()
Govt_Pincode_data['District'] = Govt_Pincode_data['District'].str.upper()
Govt_Pincode_data['State'] = Govt_Pincode_data['State'].str.upper()

Govt_Pincode_data = Govt_Pincode_data[['City','District','PIN','State']]
Govt_Pincode_data

'''


'\nGovt_Pincode_data= pd.read_csv(\'Locality_village_pincode_final_mar_2017.csv\',  encoding="ISO-8859-1")\n\nGovt_Pincode_data.rename(columns={\'Village\':\'City\'}, inplace=True)\nGovt_Pincode_data.rename(columns={\'Districtname\':\'District\'}, inplace=True)\nGovt_Pincode_data.rename(columns={\'StateName\':\'State\'}, inplace=True)\nGovt_Pincode_data.rename(columns={\'Pincode\':\'PIN\'}, inplace=True)\n\nGovt_Pincode_data[\'City\'] = Govt_Pincode_data[\'City\'].str.upper()\nGovt_Pincode_data[\'District\'] = Govt_Pincode_data[\'District\'].str.upper()\nGovt_Pincode_data[\'State\'] = Govt_Pincode_data[\'State\'].str.upper()\n\nGovt_Pincode_data = Govt_Pincode_data[[\'City\',\'District\',\'PIN\',\'State\']]\nGovt_Pincode_data\n\n'

In [0]:
#PIN_wise_coordinates= city_wise_coordinates.groupby('PIN').agg({'Lat': 'mean', 'Lng': 'mean'}).reset_index()
#PIN_wise_coordinates

In [0]:
df= pd.read_json("https://api.covid19india.org/raw_data.json")## data from covid19india.org
df3= []
for row in range(0,df.shape[0]):
    df1= df['raw_data'][row]
    df2=pd.DataFrame(df1.items()).set_index(0)
    df3.append(df2.T)
                
appended_data = pd.concat(df3, sort=False)
appended_data.replace(r'^\s*$', np.nan, regex=True, inplace = True) 
appended_data.rename(columns={'detectedcity':'City'}, inplace=True)
appended_data.rename(columns={'detecteddistrict':'District'}, inplace=True)
appended_data.rename(columns={'detectedstate':'State'}, inplace=True)

appended_data['City'] = appended_data['City'].str.lower()
appended_data['District'] = appended_data['District'].str.lower()
appended_data['State'] = appended_data['State'].str.lower()
appended_data= appended_data.dropna(thresh=3)
#appended_data


In [0]:
district_wise_counts= appended_data.groupby('District').agg({'patientnumber': 'count'})
district_wise_counts.rename(columns={'patientnumber':'d_patient_counts'}, inplace=True)
district_wise_counts =district_wise_counts.reset_index()
district_wise_counts['District'] = district_wise_counts['District'].str.upper()
#district_wise_counts

In [0]:
'''
city_wise_counts= appended_data.groupby('City').agg({'patientnumber': 'count'})
city_wise_counts.rename(columns={'patientnumber':'c_patient_counts'}, inplace=True)
city_wise_counts
'''

"\ncity_wise_counts= appended_data.groupby('City').agg({'patientnumber': 'count'})\ncity_wise_counts.rename(columns={'patientnumber':'c_patient_counts'}, inplace=True)\ncity_wise_counts\n"

In [0]:
'''

state_wise_counts= appended_data.groupby('State').agg({'patientnumber': 'count'})
state_wise_counts.rename(columns={'patientnumber':'s_patient_counts'}, inplace=True)
state_wise_counts

'''


"\n\nstate_wise_counts= appended_data.groupby('State').agg({'patientnumber': 'count'})\nstate_wise_counts.rename(columns={'patientnumber':'s_patient_counts'}, inplace=True)\nstate_wise_counts\n\n"

In [0]:
corona_db_with_latlng= district_wise_counts.merge(district_wise_data_geonames, left_on='District', right_on='District', how= 'inner')
corona_db_with_latlng.rename(columns={'d_patient_counts':'Num_Positive_cases'}, inplace=True)
#corona_db_with_latlng

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
def get_idx_distance_from_query_locations(q_lat, q_lng, corona_db_with_latlng):
    dist_array=[]
    for index, row in corona_db_with_latlng.iterrows():
        dist= int(get_distance_between_lats_lons(q_lat,q_lng,row['Lat'],row['Lng']))
        dist_array.append(dist)
    
    minpos = dist_array.index(min(dist_array)) 
    mindist= dist_array[minpos]
    cases= corona_db_with_latlng.loc[minpos,'Num_Positive_cases']
    location= corona_db_with_latlng.loc[minpos,'District']
    state= corona_db_with_latlng.loc[minpos,'State']
    Lats= corona_db_with_latlng.loc[minpos,'Lat']
    Lngs= corona_db_with_latlng.loc[minpos,'Lng']

    return(mindist, cases, location, state)

In [0]:
def get_nearest_covid19_stats(query_info,corona_db_with_latlng):
    if query_info.PIN.iloc[1] in corona_db_with_latlng['PIN'].values:
        mindist= 2
        Lat= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'Lat'];
        Lng= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'Lng'];
        mindist= int(get_distance_between_lats_lons(query_info.Lat.iloc[1] ,query_info.Lng.iloc[1], Lat,Lng))
        cases= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'Num_Positive_cases']
        district= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'District']
        state= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'State']
        print("The nearest location with COVID-19 from your PIN is in your own Postal Location with {} number of positive cases".format(cases.values[0]))
        print("Location: {} , {}".format(district.values[0].upper(), state.values[0].upper()))
    else:
        (mindist, cases, district, state) = get_idx_distance_from_query_locations(query_info.Lat.iloc[1] ,query_info.Lng.iloc[1], corona_db_with_latlng)  
        print("The nearest location with COVID-19 from your PIN is within {} km with {} number of positive cases".format(mindist, cases))
        print("Location: {} , {}".format(district.upper(), state.upper()))

In [0]:
query_pincode = input("Enter your location as an Indian PIN  : ") 
g = int(query_pincode)

if g in city_wise_coordinates.PIN.values:
    query_info= city_wise_coordinates[city_wise_coordinates.PIN == int(g)]
    get_nearest_covid19_stats(query_info,corona_db_with_latlng)
else:
    print('You entered an Invalid PIN')

Enter your location as an Indian PIN  : 826001
The nearest location with COVID-19 from your PIN is within 129 km with 3 number of positive cases
Location: MUNGER , BIHAR
